# Get Dataset from Kaggel

Install kaggle to download dataset

In [316]:
# ! pip -q install kaggle

In [317]:
import kaggle
import os
import shutil
import json

In [318]:


# # Authenticate with your Kaggle account
# kaggle.api.authenticate()

# # Details of Dataset
# dataset_id = "ayuraj/asl-dataset"
# Path = "American_Sign_Language_Dataset"
# metadata = 'dataset-metadata.json'

# # Check if metadata file exists
# if not os.path.exists(os.path.join(Path,metadata)):
#     kaggle.api.dataset_metadata(dataset_id , path=Path)

# # Open the dataset-metadata.JSON file
# with open(os.path.join(Path,metadata)) as file:
#     dataset_metadata = json.load(file)

# dataset_name = (dataset_metadata["datasetSlug"])

# # Check if dataset exists
# if not os.path.exists(os.path.join(Path,dataset_name+'.zip')):
#     # Download a dataset
#     kaggle.api.dataset_download_files(dataset, path=Path , quiet=False)



In [319]:
# # Extract the downloaded files if necessary
# import zipfile
# with zipfile.ZipFile(os.path.join(Path, dataset_name + '.zip'), 'r') as zip_ref:
#     zip_ref.extractall(Path)

# Traing Sign Language Deep Learning Model

## 1. Import and Install Dependencies

In [320]:
# ! pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [321]:
Train_Model = False
Save_Model_Path = "Test.h5"
Load_Model_Path = "action.h5"
if Train_Model:
    Load_Model_Path = Save_Model_Path

In [322]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## 2. Keypoints using MP Holistic

In [323]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [324]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [325]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

> in nwe version of holistic `FACE_CONNECTIONS` seems to be renamed/replaced by `FACEMESH_TESSELATION`

In [326]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [327]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # while cap.isOpened():
    while True:

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        if not Train_Model:
            break

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


In [328]:
draw_landmarks(frame, results)

In [329]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

## 3. Extract Keypoint Values

In [330]:
# len(results.left_hand_landmarks.landmark)
# len(results)

In [331]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [332]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [333]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [334]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [335]:
result_test = extract_keypoints(results)
result_test

array([ 0.55151236,  0.69814515, -1.32175064, ...,  0.50620252,
        0.56631112,  0.00237511])

In [336]:
np.save('0', result_test)
np.load('0.npy')

array([ 0.55151236,  0.69814515, -1.32175064, ...,  0.50620252,
        0.56631112,  0.00237511])

## 4. Setup Folders for Collection

In [337]:
doc = "Docs"
text_file = "actions.txt"
with open(os.path.join(doc, text_file), 'r') as f:
    action_str = f.read() 

# list to np array
action_names = np.array(action_str.split('.'))
actions = np.arange(len(action_names))
print(action_names)
print(actions)

['Hello' ' Good day ' ' Thank you' '']
[0 1 2 3]


In [338]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [339]:
# Remove the directory if it exists
import shutil
if os.path.exists(DATA_PATH) and Train_Model:
    shutil.rmtree(DATA_PATH)

In [340]:
# Create the directory
if Train_Model:
    os.makedirs(DATA_PATH , exist_ok=True)

    for action in actions:
        os.makedirs(os.path.join(DATA_PATH , str(action)) , exist_ok=True)


    for action in actions: 
        # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
        for sequence in range(1,no_sequences+1):
            try: 
                os.makedirs(os.path.join(DATA_PATH, str(action), str(sequence)))
            except:
                pass

## 5. Collect Keypoint Values for Training and Testing

In [341]:
cap = cv2.VideoCapture(0)

Break_capture = False
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(1, no_sequences+1):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_names[action], sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_names[action], sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, str(action), str(sequence), str(frame_num))
                
                if Train_Model:
                    np.save(npy_path, keypoints)
                else:
                    Break_capture = True
                    break
                
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    Break_capture = True
                    break
                
            if Break_capture:
                break
        if Break_capture:
            break
        
                    
    cap.release()
    cv2.destroyAllWindows()

In [342]:
cap.release()
cv2.destroyAllWindows()

## 6. Preprocess Data and Create Labels and Features

In [343]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [344]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{0: 0, 1: 1, 2: 2, 3: 3}

In [345]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, str(action)))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, str(action), str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [346]:
np.array(sequences).shape

(120, 30, 1662)

In [347]:
np.array(labels).shape

(120,)

In [348]:
X = np.array(sequences)
X.shape

(120, 30, 1662)

In [349]:
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_test.shape

(6, 4)

## 7. Build and Train LSTM Neural Network

In [350]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [351]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [352]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [353]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
if Train_Model:    
    model.fit(X_train, y_train, epochs=600, callbacks=[tb_callback])
    model.summary()

In [354]:
sign_action = np.load('MP_Data/0/1/0.npy')


## 8. Make Predictions

In [355]:
# print("X Test =\n",X_test[1])
# print("Shape" , np.array(X_test).shape)
res = model.predict(X_test)
# print("Res = \n",res)
# print("Shape" , np.array(res).shape)
actions[np.argmax(res[3])]

1/1 [==============================] - 0s 459ms/step


3

In [356]:
actions[np.argmax(y_test[4])]

0

## 9. Save Weights

In [357]:
if Train_Model:
    model.save(Save_Model_Path)
# model.save("Test.h5")


In [358]:
# del model


In [359]:
# if not Train_Model:
model.load_weights(Load_Model_Path)
# model.load_weights("action_2.h5")

## 10. Evaluation using Confusion Matrix and Accuracy

In [360]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [361]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 0s 27ms/step


array([[[3, 2],
        [0, 1]],

       [[4, 0],
        [2, 0]],

       [[2, 2],
        [2, 0]],

       [[5, 0],
        [0, 1]]])

In [362]:
accuracy_score(ytrue, yhat)

0.3333333333333333

## 11. Test in Real Time

In [363]:
from scipy import stats

In [364]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [365]:
plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

<Figure size 1800x1800 with 0 Axes>

<Figure size 1800x1800 with 0 Axes>

In [366]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        # break
        
        if len(sequence) == 30 :
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            output = action_names[np.argmax(res)]
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

        #     # Viz probabilities
        #     image = prob_viz(res, action_names, image, colors)
            
        # cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        # cv2.putText(image, ' '.join(sentence), (3,30), 
        #                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        
        # # Show to screen
        # cv2.imshow('OpenCV Feed', image)
        
        # output = str(actions[np.argmax(res)])
        # output = action_names[np.argmax(res)]
        # Add text into cv
        img = cv2.putText(image, output, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', img)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [367]:
np.array(sequences).shape

(120, 30, 1662)

In [368]:
len(sequence)

30